In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2021-11-18 04:05:37.350609: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2021-11-18 04:05:45.180645: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-18 04:05:45.221249: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-18 04:05:45.500473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 04:05:45.500741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.1

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-11-18 04:05:53.282707: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-18 04:05:53.282772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 04:05:53.283045: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.683GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2021-11-18 04:05:53.283089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 04:05:53.283278: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00

In [3]:
import numpy as np
import pandas as pd

In [4]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [5]:
# 顯示所有列
pd.set_option('display.max_columns', None)

In [ ]:
# 讀前處理過後的資料
# train = pd.read_csv('Processed_tbrain_training.csv')

# 讀合併完消費次數的看看會不會比較快
train = pd.read_csv('Processed_tbrain_training.csv')
print(train.shape)
print(train.info())
train.head(5)

In [ ]:
# 先取出每個顧客不重複最後一個月的值之後預測用
pre_train = train.drop_duplicates(subset=['chid'], keep='last')
pre_train

In [ ]:
# 去除shop_tag
pre_train.drop(columns = ['shop_tag'], axis=1, inplace=True)

In [ ]:
len(pre_train)

In [ ]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

In [ ]:
# 按照要提交的檔案排序
pre_train = pre_train.set_index('chid', drop= False)
pre_train = pre_train.loc[predict.chid]
pre_train.head(5)

In [ ]:
# 拆分測試訓練集
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

In [ ]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

In [ ]:
import gc
del train
del test
gc.collect()

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.shape[1])

In [ ]:
X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train1.shape

In [ ]:
y_train1 = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [ ]:
y_train1.shape

In [ ]:
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test1.shape

In [ ]:
y_test1 = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
y_test1.shape

In [ ]:
pre_train = pre_train.values
pre_train = np.reshape(pre_train, (pre_train.shape[0], pre_train.shape[1], 1))
pre_train.shape

In [ ]:
X_train1.shape[1]

In [ ]:
# X_train[..., None].shape

In [ ]:
# activation = 'softmax'應用於多類分類，而'sigmoid'用於二元分類。

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))

# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train1.shape[1],1 ), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(200))
# model.add(Dropout(0.2))
# model.add(Dense(49, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train[0:300].shape

In [ ]:
np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))[0:300].shape

In [ ]:
np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))[0:300].shape

In [ ]:
# tf.config.experimental.list_physical_devices('GPU')

In [ ]:
# # # from tensorflow.compat.v1 import ConfigProto
# # # from tensorflow.compat.v1 import InteractiveSession
# # # config = ConfigProto()
# # # config.gpu_options.allow_growth = True
# # # session = InteractiveSession(config=config)


# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)
    

In [ ]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
# import tensorflow as tf 
model.fit(X_train1[0:2000000], y_train1[0:2000000], epochs=1, batch_size=1, verbose=1)
# model.fit(X_train1[0:300],y_train1[0:300], validation_data = (X_test1, y_test1),
#           batch_size=1, epochs=1)

In [ ]:
# 模型預測
predictions = model.predict(pre_train)

In [ ]:
len(predictions)

In [ ]:
len(predictions[0])

In [ ]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

In [ ]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [ ]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,39,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [ ]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [ ]:
filename = 'submission-不砍欄位.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

In [ ]:
# 保存模型
model.save('LSTM_model_all-不砍欄位-1.h5') 